<a href="https://colab.research.google.com/github/FernandoIGD12/UTEC_programming101/blob/main/Examen4_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Pregunta 1

# Download data
import kagglehub

path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")
path += "/diabetes.csv"
print("Path to dataset files:", path)

# Load data into a dataframe
import pandas as pd
df_diabetes = pd.read_csv(path)
df_diabetes.head()


Path to dataset files: /kaggle/input/pima-indians-diabetes-database/diabetes.csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [27]:
print(f'Numnero de filas:{df_diabetes.shape[0]}')
print(f'Numnero de Columnas:{df_diabetes.shape[1]}')
print('\n')
print(df_diabetes.dtypes)

Numnero de filas:768
Numnero de Columnas:9


Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object


In [28]:
df_diabetes['Outcome'] = df_diabetes['Outcome'].astype('category')
print(df_diabetes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Pregnancies               768 non-null    int64   
 1   Glucose                   768 non-null    int64   
 2   BloodPressure             768 non-null    int64   
 3   SkinThickness             768 non-null    int64   
 4   Insulin                   768 non-null    int64   
 5   BMI                       768 non-null    float64 
 6   DiabetesPedigreeFunction  768 non-null    float64 
 7   Age                       768 non-null    int64   
 8   Outcome                   768 non-null    category
dtypes: category(1), float64(2), int64(6)
memory usage: 49.0 KB
None


In [29]:
# Pregunta 2
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Bajo"
    elif 18.5 <= bmi < 25:
        return "Normal"
    elif 25 <= bmi < 30:
        return "Sobrepeso"
    else:
        return "Obeso"

df_filtrado = df_diabetes[df_diabetes['Glucose'] > 120].copy()


df_filtrado['BMI_Category'] = df_filtrado['BMI'].apply(categorize_bmi)


df_filtrado.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BMI_Category
0,6,148,72,35,0,33.6,0.627,50,1,Obeso
2,8,183,64,0,0,23.3,0.672,32,1,Normal
4,0,137,40,35,168,43.1,2.288,33,1,Obeso
8,2,197,70,45,543,30.5,0.158,53,1,Obeso
9,8,125,96,0,0,0.0,0.232,54,1,Bajo


In [51]:
# Pregunta 3

df_age_groups = pd.read_csv('age_groups.csv')
df_combinado = pd.merge(df_diabetes, df_age_groups, on='Age', how='left')
df_combinado.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,AgeGroup
0,6,148,72,35,0,33.6,0.627,50,1,Adulto
1,1,85,66,29,0,26.6,0.351,31,0,Adulto
2,8,183,64,0,0,23.3,0.672,32,1,Adulto
3,1,89,66,23,94,28.1,0.167,21,0,Joven
4,0,137,40,35,168,43.1,2.288,33,1,Adulto


In [52]:
# Pregunta 4

# Agrupamiento por AgeGroup y calculo de promedio de Glucose y BMI
grouped_glocuse = df_combinado.groupby('AgeGroup')['Glucose'].mean()
grouped_BMI = df_combinado.groupby('AgeGroup')['BMI'].mean()
grouped = pd.merge(grouped_glocuse, grouped_BMI, on='AgeGroup', how='outer')
# Ordenamiento por Glucose descendiente
grouped_sorted = grouped.sort_values(by = 'Glucose', ascending=False)

grouped_sorted.head()

,Glucose,BMI
AgeGroup,,
Adulto_Mayor,139.679012,30.211111
Adulto,125.637037,33.562593
Joven,114.175060,31.322062


In [62]:
# Pregunta 5

# 1. Filtra adultos diabeticos
adultos_diabeticos = df_combinado[(df_combinado['AgeGroup'] == 'Adulto') & (df_combinado['Outcome'] == 1)]

# 2. Obtenemos los 5 registros mas altos de BMI
top_bmi = adultos_diabeticos.nlargest(5, 'BMI')


print(top_bmi[['Age', 'Glucose', 'BMI', 'DiabetesPedigreeFunction']].sort_values('BMI', ascending=False))

     Age  Glucose   BMI  DiabetesPedigreeFunction
193   40      135  52.3                     0.578
155   36      152  50.0                     0.337
99    31      122  49.7                     0.325
84    37      137  48.8                     0.227
378   32      156  48.3                     0.238


In [63]:
# Pregunta 6

pivot_table = pd.pivot_table(df_combinado, index='AgeGroup', columns='Outcome', values=['Glucose', 'BMI'], aggfunc='mean')
filtered = pivot_table[(pivot_table[('BMI', 0)] > 30) | (pivot_table[('BMI', 1)] > 30)]
print(filtered)

                    BMI                Glucose            
Outcome               0          1           0           1
AgeGroup                                                  
Adulto        31.923077  35.085000  111.830769  138.457143
Adulto_Mayor  28.413953  32.244737  127.209302  153.789474
Joven         29.909174  36.455556  106.978593  140.322222
